In [62]:
from scipy.io import loadmat
import numpy as np

#LASSO Solver - provided code in earlier assignment
def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X

#################################################################
#SETUP
X = loadmat("RawData.mat")['X']
y = loadmat("RawData.mat")['y']
Xones = np.ones((len(X),1))

#Optional: Eliminate high values
#for i in range(110):
#    X = np.delete(X, np.argmax(y), 0)
#    y = np.delete(y, np.argmax(y), 0)
#    X = np.delete(X, np.argmin(y), 0)
#    y = np.delete(y, np.argmin(y), 0)

TwoNormCol = np.zeros((len(X.T),1))

#Remove keyword columns (poorly treated data)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)

#Normalize columns to 2-norm
for i in range(len(X.T)):
    TwoNormCol[i] = np.sqrt(X[i,:]@X[i,:])
    X[i,:] = X[i,:]/TwoNormCol[i]
#print(TwoNormCol)
#print(X[0,:])
#print(y)
    
#Form subsets (indices, first group is full X)
Xsubs = np.array([[0,49],[0,5],[5,7],[7,9],[9,11],[11,17],[17,20],[20,26],[28,33],[33,37],[37,45],[45,49]])

#Create and run over 11 sets of 3604 entries from X and y,
#using 1 as a primary test set (when needed for lambda)
setArr = np.array([[0,int(len(X)/11)],[int(len(X)/11),int(2*len(X)/11)],[int(2*len(X)/11),int(3*len(X)/11)],\
                   [int(3*len(X)/11),int(4*len(X)/11)],[int(4*len(X)/11),int(5*len(X)/11)],\
                   [int(5*len(X)/11),int(6*len(X)/11)],[int(6*len(X)/11),int(7*len(X)/11)],\
                   [int(7*len(X)/11),int(8*len(X)/11)],[int(8*len(X)/11),int(9*len(X)/11)],\
                   [int(9*len(X)/11),int(10*len(X)/11)],[int(10*len(X)/11),int(len(X))]])
#print(setArr)

#Set up lambda
lambdaCount = 20
lambdaTest = 5*np.logspace(-6, 12, lambdaCount)

#Error tally storage
errorOrig = np.zeros((12,1))
#errorOrigOnes = np.zeros((11,1))
errorLASSO = np.zeros((12,1))
#errorRidge = np.zeros((11,1))


################################3

#Loops over (1) all X subsets, (2 and 3) over all 11 sets for testing (1320 loops total)
for Xset in range(len(Xsubs)):
    for i in range(11):
        for j in range(11):
            if i != j:
                print("X subset = ",Xset,", i = ",i,", j = ",j)
                testTally = 0
                #Set up Training and testing sets
                for k in range(11):
                    if i == k:
                        XTest1 = X[setArr[i,0]:setArr[i,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                        #print(XTest1)
                        yTest1 = y[setArr[i,0]:setArr[i,1]]
                        #print(yTest1)
                    if j == k:
                        XTest2 = X[setArr[j,0]:setArr[j,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                        #print(XTest2)
                        yTest2 = y[setArr[j,0]:setArr[j,1]]
                        #print(yTest2)
                    if k != j and k != i:
                        if testTally == 0:
                            XTrain = X[setArr[k,0]:setArr[k,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                            yTrain = y[setArr[k,0]:setArr[k,1]]
                            testTally = 1
                        else:
                            XTrain = np.concatenate((XTrain, X[setArr[k,0]:setArr[k,1],Xsubs[Xset,0]:Xsubs[Xset,1]]),\
                                                   axis=0)
                            yTrain = np.concatenate((yTrain, y[setArr[k,0]:setArr[k,1]]), axis=0)
                #print(len(XTrain))
                #print(len(XTrain.T))
                
                #Add a column of ones for comparison
                Xones = np.ones((len(XTrain),1))
                XTrainwithOne = np.concatenate((XTrain, Xones), axis=1)
                Xones2 = np.ones((len(XTest2),1))
                XTest2withOne = np.concatenate((XTest2, Xones2), axis=1)
                
                #Additional polynomial tests
                #UPDATE 11/13: SOME RESULT IN SINGULAR MATRICES, NOT TESTING FURTHER
                #Xpoly2 = np.concatenate((XTrain, XTrain*XTrain, Xones), axis=1)
                #Xpoly3 = np.concatenate((XTrain, XTrain*XTrain, XTrain*XTrain*XTrain, Xones), axis=1)
                
                #Training 1 (for sets that need it)
                #Lowest error is based on lowest average differences (abs val) between y and Xw
                WLASSO = ista_solve_hot(XTrain,yTrain,lambdaTest)
    
                # Storage for current iteration ridge regression
                #Wrid = np.zeros((len(X.T),lambdaCount))
                
                #UPDATE 11/15: Has been too much for the kernel to handle (using optimized setup per activity 17)
                #Not continuing with Ridge Regression
                # Ridge regression
                #for index in range(lambdaCount):
                    #WRID[:,index] = np.linalg.inv(XTrain.T@XTrain+lambdaTest[index]*\
                                                  #np.identity(len(XTrain)))@XTrain.T@yTrain[:,0]
    
                #check with Test Set 1
                y_lasso1 = XTest1@WLASSO
                #y_rid1 = XTest1@WRID
                errorTallyLasso = np.zeros((lambdaCount,1))
                #errorTallyRid = np.zeros((lambdaCount,1))
                for lam in range(lambdaCount):
                    for index in range(len(XTest1)):
                        errorTallyLasso[lam] = errorTallyLasso[lam] + abs(yTest1[index]-y_lasso1[index,lam])\
                        /yTest1[index]
                        #errorTallyRid[lam] = errorTallyRid + abs(yTest1[index]-y_rid1[index])
                    #print(errorTallyLasso[lam]/(len(X)/11))

                #print(errorTallyLasso)
                #print(errorTallyRid)
                OptimalLambdaLasso = np.argmin(errorTallyLasso)
                #print("Optimal lambda: ",lambdaTest[OptimalLambdaLasso]," with min ",min(errorTallyLasso)/(len(X)/11))
                #print(np.argmin(errorTallyLasso))
                #OptimalLambdaRid = np.argmin(errorTallyRid)
                #print(np.argmin(errorTallyRid))
                
                #Running on Testing 2
                wOrig = np.linalg.inv(XTrain.T@XTrain)@XTrain.T@yTrain
                yTestOrig = XTest2@wOrig
                #print(yTestOrig)
                yTestLASSO = XTest2@WLASSO[:,OptimalLambdaLasso]

                errorOrigTemp = 0
                errorLASSOTemp = 0
                for index in range(len(XTest2)):
                    errorOrigTemp = errorOrigTemp + abs(yTest1[index]-yTestOrig[index])/yTest1[index]
                    errorLASSOTemp = errorLASSOTemp + abs(yTest1[index]-yTestLASSO[index])/yTest1[index]
                
                errorOrigTemp = errorOrigTemp/len(XTest2)
                errorLASSOTemp = errorLASSOTemp/len(XTest2)
                print("Mean error from standard least squares: ",errorOrigTemp)
                print("Mean error from LASSO: ",errorLASSOTemp)
                
                errorOrig[Xset] = errorOrig[Xset] + errorOrigTemp
                errorLASSO[Xset] = errorLASSO[Xset] + errorLASSOTemp
                
                #UPDATE 11/13: SOME ONES X RESULT IN SINGULAR MATRIX, NOT TESTING FURTHER
                #wOrigOnes = np.linalg.inv(XTrainwithOne.T@XTrainwithOne)@XTrainwithOne.T@yTrain
                #yTestOrigOne = XTest2withOne@wOrigOnes
                #print(yTestOrigOne)

GlobalMeanOrigError = np.zeros((11,1))
GlobalMeanLASSOError = np.zeros((11,1))
for index in range(11):
    GlobalMeanOrigError[index] = errorOrig[index]/110
    GlobalMeanLASSOError[index] = errorLASSO[index]/110
    print("Mean Error for pure least squares, X subset ",(index+1)," = ",GlobalMeanOrigError[index])
    print("Mean Error for LASSO, X subset ",(index+1)," = ",GlobalMeanLASSOError[index])





X subset =  0 , i =  0 , j =  1
Mean error from standard least squares:  [2.87405386]
Mean error from LASSO:  [0.94831663]
X subset =  0 , i =  0 , j =  2
Mean error from standard least squares:  [8.25789016]
Mean error from LASSO:  [0.91782115]
X subset =  0 , i =  0 , j =  3
Mean error from standard least squares:  [22.22399951]
Mean error from LASSO:  [0.88043183]
X subset =  0 , i =  0 , j =  4
Mean error from standard least squares:  [2.69954955]
Mean error from LASSO:  [0.9274099]
X subset =  0 , i =  0 , j =  5
Mean error from standard least squares:  [7.19580145]
Mean error from LASSO:  [0.89175439]
X subset =  0 , i =  0 , j =  6
Mean error from standard least squares:  [5.25253998]
Mean error from LASSO:  [0.99420246]
X subset =  0 , i =  0 , j =  7
Mean error from standard least squares:  [2.36663485]
Mean error from LASSO:  [0.96640917]
X subset =  0 , i =  0 , j =  8
Mean error from standard least squares:  [43.74057366]
Mean error from LASSO:  [0.89942192]
X subset =  0 ,